In [1]:
import os
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

## Completion Model

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-20b")

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {objective} story about {topic}"
)

llmmodelprompt = prompt_template.format(
    objective="curious",
    topic="Tesla"
)

res = model.invoke(llmmodelprompt)
print(res.content)



### The Day Tesla Heard Himself in New York

In the summer of 1899, when the world was still buzzing with the electric hum of early dynamos, Nikola Tesla was on a mission that would push the boundaries of physics and leave a curious breadcrumb in history.

Tesla had built a 100‑foot “tower” in the high desert of Colorado Springs—an enormous copper‑clad structure with a 20,000‑volt Tesla coil at its apex. The goal was simple in theory, impossible in practice: transmit electricity without wires over long distances. He wanted to power a city from a single point, a dream of a world where the grid would be invisible.

On a clear night, Tesla turned the coil’s iron core into a roaring furnace of sparks. The air crackled, a faint hiss of ionized air filled the canyon, and a low, almost musical tone resonated through the tower. The experiment was a success—electricity flowed from the coil, lighting a nearby lamp that was 40 miles away in Pueblo, Colorado. Tesla was ecstatic; the world was one 

In [9]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Provide the Helpful response to the user about the {topic}"),
        ("user", "{user_input}")
    ]
)

messages = chat_template.format_messages(
    topic="Country",
    user_input="What is the capital of France?"
)

response = model.invoke(messages)
print(response.content)


The capital of France is **Paris**.


## Fewshots Example

In [26]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

# define the examples
examples = [
    {"user": "hi!", "assistant": "¡hola!"},
    {"user": "bye!", "assistant": "¡adiós!"},
]

# define the example prompt template
chat_template = ChatPromptTemplate.from_messages(
    [
    ("user", "{user}"),
    ("assistant", "{assistant}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=chat_template,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "Provide the helpful response to the user."),
    few_shot_prompt,
    ("user", "{user_input}")
    ]
)

input_message = final_prompt.format_messages(
    user_input="Tell me a joke about tom and jerry cartoon."
)

response = model.invoke(input_message)
print(response.content)

Why did Tom bring a ladder to the kitchen?

Because he heard the mouse was “high‑spirited” and wanted to catch Jerry on a **higher level**! 🐱🧗‍♂️🐭


Output With Chains

In [30]:
final_prompt

ChatPromptTemplate(input_variables=['user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Provide the helpful response to the user.')), FewShotChatMessagePromptTemplate(examples=[{'user': 'hi!', 'assistant': '¡hola!'}, {'user': 'bye!', 'assistant': '¡adiós!'}], example_prompt=ChatPromptTemplate(input_variables=['assistant', 'user'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user'], template='{user}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['assistant'], template='{assistant}'))])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [31]:
chain = final_prompt | model
response = chain.invoke({"user_input":"Tell me a joke about tom and jerry cartoon."})
print(response.content)

Why did Tom bring a ladder to the kitchen?

Because he heard the mouse was “on a higher level” and wanted to **climb** the competition! 🐱🦠🤣


In [35]:
response.response_metadata

{'token_usage': {'completion_tokens': 67,
  'prompt_tokens': 124,
  'total_tokens': 191,
  'completion_time': 0.066314232,
  'prompt_time': 0.006164083,
  'queue_time': 0.086744162,
  'total_time': 0.072478315},
 'model_name': 'openai/gpt-oss-20b',
 'system_fingerprint': 'fp_77f8660d1d',
 'finish_reason': 'stop',
 'logprobs': None}

In [37]:
response.usage_metadata

{'input_tokens': 124, 'output_tokens': 67, 'total_tokens': 191}

## Output Parser

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser
response  = json_chain.invoke({"question": "What is the biggest country?"})
response


{'answer': 'Russia'}

## Defining the Pydantic Custom Output Parser

In [43]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class Joke( BaseModel ):
    setup: str = Field(description="Question to make the Joke")
    response: str = Field(descrption="Answer to create the Joke")

parser = JsonOutputParser(pydantic_object=Joke)


In [45]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "Question to make the Joke", "type": "string"}, "response": {"title": "Response", "descrption": "Answer to create the Joke", "type": "string"}}, "required": ["setup", "response"]}\n```'

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | model | parser

chain.invoke({"query": "Tell me a joke about tom and jerry cartoon."})

{'setup': 'Why did Tom bring a ladder to the cartoon studio?',
 'punchline': 'Because Jerry was always on the high shelf of mischief!'}

: 